# Scrape IDEO Journal

In [1]:
# Jupyter magic
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [31]:
# imports
import requests
from bs4 import BeautifulSoup
import re
import time
import random
import pandas as pd
# from seleniumrequests import Chrome
import pathlib

In [33]:
save_dir = "../1_data/IDEO_journal"
pathlib.Path(save_dir).exists()

True

## Create index of URLs for journal posts

In [17]:
url = "https://www.ideo.com/journal"


response = requests.get(url)
print(response)
soup = BeautifulSoup(response.text, 'html.parser')
links1 = [a.get('href') for a in soup.find_all('a')]
len(links1)

<Response [200]>


79

In [18]:
url = "https://www.ideo.com/journal/archive/2"

response = requests.get(url)
print(response)
soup = BeautifulSoup(response.text, 'html.parser')
links2 = [a.get('href') for a in soup.find_all('a')]
len(links2)

<Response [200]>


109

In [19]:
unique_links = links1 + links2
print(f'before dedupe: {len(unique_links)}')
unique_links = list(set(unique_links))
len(unique_links)

before dedupe: 188


72

In [46]:
article_links = [l for l in unique_links if '/journal/' in l]
article_links.remove('/journal/archive')

full_urls = ['https://www.ideo.com' + link for link in article_links]


len(full_urls)

35

In [47]:
# Save them as backup
with open(f'{save_dir}/index.txt', 'w') as f:
    f.writelines(full_urls)

## Grab the text for each post

In [50]:
full_urls[0]

'https://www.ideo.com/journal/the-best-business-breakthroughs-come-from-moments-of-doubt'

In [49]:
article_url = full_urls[0]
response = requests.get(full_url)
print(response)
soup = BeautifulSoup(response.text, 'html.parser')


<Response [200]>


In [45]:
full_url

'https://www.ideo.com/journal/the-best-business-breakthroughs-come-from-moments-of-doubt'

In [42]:
title = soup.find('h1').text


'The Best Business Breakthroughs Come From Moments of Doubt'

In [60]:
junk = [
'Send this to a friend:',
 'Related Posts',
 'Send this to a friend:',
 'Say Hello.',
 'Come work with us! We are always looking for great talent to join our global teams.',
'Send this to a friend:',
]

def get_article(article_url):
    response = requests.get(article_url)
    print(response)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('h1').text
    text = [t.text for t in soup.find_all(['p', 'h2'])]
    
    for j in junk:
        text.remove(j)
    
    

    
    return title, text

title, text = get_article(full_url)


<Response [200]>


In [62]:
for article_url in full_urls:
    title, text = get_article(article_url)

    print(title)
    print(len(text), 'lines')
    
    fname = f"{save_dir}/{title}.txt"
    print(fname)
    with open(fname, 'w') as f:
        f.writelines(text)

<Response [200]>
The Best Business Breakthroughs Come From Moments of Doubt
34 lines
../1_data/IDEO_journal/The Best Business Breakthroughs Come From Moments of Doubt.txt'
<Response [200]>
5 Leaders Reflect on How the Pandemic Has Reshaped Their Thinking
32 lines
../1_data/IDEO_journal/5 Leaders Reflect on How the Pandemic Has Reshaped Their Thinking.txt'
<Response [200]>
Innovation Thrives Under These 5 Evidence-Backed Conditions
16 lines
../1_data/IDEO_journal/Innovation Thrives Under These 5 Evidence-Backed Conditions.txt'
<Response [200]>
Prototype the Future of Your Business With This 4-Step Design Exercise
27 lines
../1_data/IDEO_journal/Prototype the Future of Your Business With This 4-Step Design Exercise.txt'
<Response [200]>
Company Say, Company Do: How Your Words Define Your Business
5 lines
../1_data/IDEO_journal/Company Say, Company Do: How Your Words Define Your Business.txt'
<Response [200]>
Resilience in Uncertain Times: Insights From Global Leaders
23 lines
../1_data/I

In [66]:
# for n in pathlib.Path(save_dir).glob('*'):
#     n.rename(n.name.strip("'"))